# Assignment 1. Creating Objects 
## Gema Castillo García

### Files I have created:
Scripts defining each Class:

    Gene.rb
    Seed_stock.rb
    Cross.rb

**The 3 Classes are interconnected. The values of some Object Properties are other Objects: the Gene ID is the key to link Gene > Seed_stock and the seed stock ID is the key to link Seed_stock > Cross_data**. 


The main script that uses the previous 3 scripts to update the genebank information and to determine which genes are
genetically-linked is:

    process_database.rb


### Final program
    $ ruby process_database.rb  gene_information.tsv  seed_stock_data.tsv  cross_data.tsv new_stock_file.tsv
    
    
    

In [1]:
#the code of this box is in Gene.rb file

#beginning a class definition for gene_information.tsv data
class Gene
  #creating "attribute accessors" to read and write objects' properties
  attr_accessor :id
  attr_accessor :name
  attr_accessor :mutant_phenotype
  attr_accessor :linked_to
  
  #initialization inside the class
  def initialize(id ,name ,mutant_phenotype , linked_to) 
    @id = id
    @name = name
    @mutant_phenotype = mutant_phenotype
    @linked_to = linked_to
  end

  
  def Gene.load_from_file() ##BONUS: the object have a #load_from_file($gene_information.tsv)
    require 'csv'
    gene_info = [] #creating a new array that will contain the 'gene_information.tsv' data without the header by rows
    CSV.foreach('gene_information.tsv', headers: true, :quote_char => "|") do |row|
      gene_info << row
    end
    
    #creating objects for 'Gene Class' and passing parameters for initialization
    @@all_genes = [] 
    for $x in 0..4
      if $x==0
        #formatting the information contained in the array with the 'gene_information.tsv' data
        gene_infox = gene_info[$x][0].split("\t")
        gene_infox.map!{ |element| element.gsub(/"/, '') }
        if gene_infox[0] =~ /A[Tt]\d[Gg]\d\d\d\d\d/ ##BONUS: testing the format of the Gene ID 
          @@all_genes << Gene.new(gene_infox[0],gene_infox[1],gene_infox[2],"none") #creating an array with the new Objects
        else
          puts "The Gene ID is not correct. Please, check it!" ##BONUS: rejecting incorrect Gene ID without crashing
        end
      else
        #formatting the information contained in the array with the 'gene_information.tsv' data
        gene_infox = "#{gene_info[$x][0]}, #{gene_info[$x][1]}"
        gene_infox = gene_infox.split("\t")
        gene_infox.map!{ |element| element.gsub(/"/, '') }
        if gene_infox[0] =~ /A[Tt]\d[Gg]\d\d\d\d\d/ ##BONUS: testing the format of the Gene ID 
          @@all_genes << Gene.new(gene_infox[0],gene_infox[1],gene_infox[2],"none") #creating an array with the new Objects
        else
          puts "The Gene ID is not correct. Please, check it!" ##BONUS: rejecting incorrect Gene ID without crashing
        end
        $x +=1
      end
    end
    puts @@all_genes #showing the content of the created objects for 'Gene Class'
    puts
  end
  
  
  def Gene.show_genes #defining a method to work with Gene Class objects
    return @@all_genes
  end
  

  def Gene.linked_genes #defining a method to write a final report with linked genes
    puts "Final Report:"
    Gene.show_genes.each do |g|
      if g.linked_to != "none"
        puts "#{g.name} is linked to #{g.linked_to}"
      end
    end
  end
  
end


:linked_genes

In [2]:
#the code of this box is in Seed_stock.rb file

#beginning a class definition for seed_stock_data.tsv information
class Seed_stock < Gene #Seed_stock is inheriting the properties of Gene
  #creating "attribute accessors" to read and write objects' properties
  attr_accessor :stock
  attr_accessor :gene_ID
  attr_accessor :last_planted
  attr_accessor :storage
  attr_accessor :grams_remaining
  
  
  #initialization inside the class
  def initialize(stock ,gene_ID ,last_planted ,storage ,grams_remaining ) 
    @stock = stock    
    @gene_ID = gene_ID    
    @last_planted = last_planted
    @storage = storage
    @grams_remaining = grams_remaining
  end


  def Seed_stock.load_from_file() ##BONUS: the object have a #load_from_file($seed_stock_data.tsv)
    require 'csv'
    seed_stock_data = [] #creating a new array that will contain the 'seed_stock_data.tsv' data without the header by rows
    CSV.foreach('seed_stock_data.tsv', headers: true, :quote_char => "|") do |row|
      seed_stock_data << row
    end
    
    @@all_stocks = []
    for $s in 0..4
      seed_stock_datax = seed_stock_data[$s][0].split("\t") #formatting the information contained in the array with the 'seed_stock_data.tsv' data
      @@all_stocks << Seed_stock.new(seed_stock_datax[0],seed_stock_datax[1],seed_stock_datax[2],seed_stock_datax[3],seed_stock_datax[4]) #creating an array with the new Objects
      $s +=1
    end
    
    #introducing a Gene's Object in gene_ID property value by Gene ID (ATxGgxxxxx)
    Gene.show_genes.each do |y|
      @@all_stocks.each do |z|
        if y.id == z.gene_ID
        z.gene_ID = y
        #each Seed_stock Object will have the corresponding Gene's Object in its property @gene_ID
        end
      end
    end
    puts @@all_stocks #showing the content of the created objects for 'Seed_stock Class'
    puts
  end
  
  
  def Seed_stock.show_stocks #defining a method to work with Seed_stock Class objects
    return @@all_stocks
  end
  
  
  def Seed_stock.updating_seed_stock(grams_to_plant, date) #defining a method to update the amount of remaining grams in a specific date
    ##BONUS: the object have a #write_database('new_stock_file.tsv')
    Seed_stock.show_stocks.each do |s|
    updated_grams = s.grams_remaining.to_i - grams_to_plant
    s.last_planted = date
      if updated_grams <= 0
        updated_grams = 0
        puts "WARNING: we have run out of Seed Stock #{s.stock}"
        File.open("new_stock_file.tsv", "a") { |f| f.write("#{s.stock}\t#{s.gene_ID.id}\t#{s.last_planted}\t#{s.storage}\t#{updated_grams}\n") }
      else
        File.open("new_stock_file.tsv", "a") { |f| f.write("#{s.stock}\t#{s.gene_ID.id}\t#{s.last_planted}\t#{s.storage}\t#{updated_grams}\n") }
      end
    end
  end
  
  
  def Seed_stock.get_seed_stock(somestock) ##BONUS: objects may access individual SeedStock objects based on their stock ID (e.g., 'A334')
    Seed_stock.show_stocks.each do |s|
      if s.stock.include?(somestock)
        puts "Stock ID: #{s.stock}, Gene ID: #{s.gene_ID.id}, last planted: #{s.last_planted}, storage: #{s.storage}, grams remaining: #{s.grams_remaining}"
        break
      else
        puts "The Stock ID #{somestock} is not present in the database."
        break
      end
    end
  end
  
end

:get_seed_stock

In [3]:
#the code of this box is in Cross.rb file

#beginning a class definition for cross_data.tsv information
class Cross < Seed_stock #Cross is inheriting the properties of Seed_stock
  #creating "attribute accessors" to read and write objects' properties
  attr_accessor :parent_1
  attr_accessor :parent_2
  attr_accessor :f2_wild
  attr_accessor :f2_P1
  attr_accessor :f2_P2
  attr_accessor :f2_P1P2
  
  #initialization inside the class
  def initialize(parent_1 ,parent_2 ,f2_wild ,f2_P1 ,f2_P2 ,f2_P1P2 ) 
    @parent_1 = parent_1
    @parent_2 = parent_2
    @f2_wild = f2_wild
    @f2_P1 = f2_P1
    @f2_P2 = f2_P2
    @f2_P1P2 = f2_P1P2
  end
  
  
  def Cross.load_from_file() ##BONUS: the object have a #load_from_file($seed_stock_data.tsv)
    require 'csv'
    cross_data = [] #creating a new array that will contain the 'cross_data.tsv' data without the header by rows
    CSV.foreach('cross_data.tsv', headers: true, :quote_char => "|") do |row|
      cross_data << row
    end
    
    #creating objects for 'Cross Class' and passing parameters for initialization
    @@all_crosses = []
    for $c in 0..4
      cross_datax = cross_data[$c][0].split("\t") #formatting the information contained in the array with the 'cross_data.tsv' data
      @@all_crosses << Cross.new(cross_datax[0],cross_datax[1],cross_datax[2],cross_datax[3],cross_datax[4],cross_datax[5]) #creating an array with the new Objects
      $c +=1
    end
    
    #introducing a Seed_stock's Object in parent_1/2 properties values by seed stock ID (e.g., A348)
    Seed_stock.show_stocks.each do |yy|
      @@all_crosses.each do |zz|
        if yy.stock == zz.parent_1
          zz.parent_1 = yy
          #each Cross Object will have the corresponding Seed_stock's Object in its property @parent_1
        end
        if yy.stock == zz.parent_2
          zz.parent_2 = yy
          #each Cross Object will have the corresponding Seed_stock's Object in its property @parent_2
        end
      end
    end
    puts @@all_crosses #showing the content of the created objects for 'Cross Class'
    puts
  end

  
  def Cross.show_crosses #defining a method to work with Cross Class objects
    return @@all_crosses
  end
 
  
  def Cross.chi_square #defining a method to do the chi² test
    Cross.show_crosses.each do |c|
      total_F2 = c.f2_wild.to_i + c.f2_P1.to_i + c.f2_P2.to_i + c.f2_P1P2.to_i
      expected = [total_F2*(9.0/16), total_F2*(3.0/16), total_F2*(3.0/16), total_F2*(1.0/16)] #expected frequencies if not linked for dihybrids' (heterozygous for 2 characters) offspring
      chi² = ((((c.f2_wild.to_i - expected[0])**2)/expected[0]) + (((c.f2_P1.to_i - expected[1])**2)/expected[1]) + (((c.f2_P2.to_i - expected[2])**2)/expected[2]) + (((c.f2_P1P2.to_i - expected[3])**2)/expected[3]))
      if chi² > 7.8147 #results are statistically significant (p < 0.05) with df = 3 (4 phenotypes - 1)        
        c.parent_1.gene_ID.linked_to = c.parent_2.gene_ID.name
        c.parent_2.gene_ID.linked_to = c.parent_1.gene_ID.name
        puts "Recording: #{c.parent_1.gene_ID.name} is genetically linked to #{c.parent_2.gene_ID.name} with chisquare score #{chi²}"
      end
    end
  end
  
end


##BONUS: this Class represents the entire Seed Stock "database"

:chi_square

## 1) "Simulate" planting 7 grams of seeds from each of the records in the seed stock genebank, then update the genebank information to show the new quantity of seeds that remain after a planting
- The new state of the genebank should be printed to a new file (new_stock_file.tsv), using exactly the same format as the original file seed_stock_data.tsv
- If the amount of seed is reduced to zero or less than zero, then a friendly WARNING message should appear on the screen
> The amount of seed left in the gene bank is, of course, not LESS than zero

In [ ]:
#only run if you haven't run the boxes above
require './Gene.rb'  #using Gene Class
require './Seed_stock.rb'  #using Seed_stock Class
require './Cross.rb'  #using Cross Class

In [4]:
#creating objects from tsv files and showing their content (puts @@all_stocks/genes/crosses in boxes above)
##this output is not visible in the terminal because the .rb files has '##' before puts @@all_stocks/genes/crosses

Gene.load_from_file() #all objects with linked_to="none" because "none" is the default value
Seed_stock.load_from_file() #Seed_stock has inherited Gene objects in the @gene_ID property
Cross.load_from_file() #Cross has inherited Seed_stock objects in @parent_1 and @parent_2 properties

[#<Gene:0x000055ee866efaa8 @id="AT1G69120", @name="ap1", @mutant_phenotype="meristems replace first and second whorl", @linked_to="none">, #<Gene:0x000055ee866ef698 @id="AT4G36920", @name="ap2", @mutant_phenotype="first whorl carpels,  second whorl stamens", @linked_to="none">, #<Gene:0x000055ee866ef288 @id="AT3G54340", @name="ap3", @mutant_phenotype="second whorl sepals,  third whorl carpels", @linked_to="none">, #<Gene:0x000055ee866eee78 @id="AT1G30950", @name="ufo", @mutant_phenotype="second whorl sepaloidy,  third whorl missing or carpeloid", @linked_to="none">, #<Gene:0x000055ee866eea68 @id="AT5G20240", @name="pi", @mutant_phenotype="second whorl sepals,  third whorl carpels", @linked_to="none">]

[#<Seed_stock:0x000055ee866f6600 @stock="A334", @gene_ID=#<Gene:0x000055ee866efaa8 @id="AT1G69120", @name="ap1", @mutant_phenotype="meristems replace first and second whorl", @linked_to="none">, @last_planted="5/7/2014", @storage="cama2", @grams_remaining="28">, #<Seed_stock:0x000055ee86

In [5]:
##BONUS: the object have a #write_database('new_stock_file.tsv')
File.open("new_stock_file.tsv", "a") { |f| f.write("Seed_Stock\tMutant_Gene_ID\tLast_Planted\tStorage\tGrams_Remaining\n") }
Seed_stock.updating_seed_stock(7, "16/10/2021")

[#<Seed_stock:0x000055ee866f6600 @stock="A334", @gene_ID=#<Gene:0x000055ee866efaa8 @id="AT1G69120", @name="ap1", @mutant_phenotype="meristems replace first and second whorl", @linked_to="none">, @last_planted="16/10/2021", @storage="cama2", @grams_remaining="28">, #<Seed_stock:0x000055ee866f6420 @stock="A348", @gene_ID=#<Gene:0x000055ee866ef698 @id="AT4G36920", @name="ap2", @mutant_phenotype="first whorl carpels,  second whorl stamens", @linked_to="none">, @last_planted="16/10/2021", @storage="cama25", @grams_remaining="12">, #<Seed_stock:0x000055ee866f6240 @stock="B3334", @gene_ID=#<Gene:0x000055ee866ef288 @id="AT3G54340", @name="ap3", @mutant_phenotype="second whorl sepals,  third whorl carpels", @linked_to="none">, @last_planted="16/10/2021", @storage="cama18", @grams_remaining="22">, #<Seed_stock:0x000055ee866f6060 @stock="A51", @gene_ID=#<Gene:0x000055ee866eee78 @id="AT1G30950", @name="ufo", @mutant_phenotype="second whorl sepaloidy,  third whorl missing or carpeloid", @linked_to=

In [6]:
##BONUS: objects may access individual SeedStock objects based on their stock ID
##(I haven't put this on the process_database.rb file because it is not required in the instructions)
Seed_stock.get_seed_stock("A334") #if you search for a stock ID present in your Class, you obtain the corresponding object
Seed_stock.get_seed_stock("B334") #if you search for a stock ID not present in your Class, you obtain a warning message

Stock ID: A334, Gene ID: AT1G69120, last planted: 16/10/2021, storage: cama2, grams remaining: 28
The Stock ID B334 is not present in the database.


## 2)	Process the information in cross_data.tsv and determine which genes are genetically-linked.
To achieve this, you will have to do a Chi-square test on the F2 cross data. If you discover genes that are linked, this information should be added as a property of each of the genes (they are both linked to each other).

In [7]:
Cross.chi_square

Recording: ufo is genetically linked to pi with chisquare score 32.27942794279428


[#<Cross:0x000055ee86701848 @parent_1=#<Seed_stock:0x000055ee866f6600 @stock="A334", @gene_ID=#<Gene:0x000055ee866efaa8 @id="AT1G69120", @name="ap1", @mutant_phenotype="meristems replace first and second whorl", @linked_to="none">, @last_planted="16/10/2021", @storage="cama2", @grams_remaining="28">, @parent_2=#<Seed_stock:0x000055ee866f6420 @stock="A348", @gene_ID=#<Gene:0x000055ee866ef698 @id="AT4G36920", @name="ap2", @mutant_phenotype="first whorl carpels,  second whorl stamens", @linked_to="none">, @last_planted="16/10/2021", @storage="cama25", @grams_remaining="12">, @f2_wild="110", @f2_P1="33", @f2_P2="38", @f2_P1P2="11">, #<Cross:0x000055ee86701640 @parent_1=#<Seed_stock:0x000055ee866f6420 @stock="A348", @gene_ID=#<Gene:0x000055ee866ef698 @id="AT4G36920", @name="ap2", @mutant_phenotype="first whorl carpels,  second whorl stamens", @linked_to="none">, @last_planted="16/10/2021", @storage="cama25", @grams_remaining="12">, @parent_2=#<Seed_stock:0x000055ee866f6240 @stock="B3334", @

In [8]:
Gene.linked_genes

Final Report:
ufo is linked to pi
pi is linked to ufo


[#<Gene:0x000055ee866efaa8 @id="AT1G69120", @name="ap1", @mutant_phenotype="meristems replace first and second whorl", @linked_to="none">, #<Gene:0x000055ee866ef698 @id="AT4G36920", @name="ap2", @mutant_phenotype="first whorl carpels,  second whorl stamens", @linked_to="none">, #<Gene:0x000055ee866ef288 @id="AT3G54340", @name="ap3", @mutant_phenotype="second whorl sepals,  third whorl carpels", @linked_to="none">, #<Gene:0x000055ee866eee78 @id="AT1G30950", @name="ufo", @mutant_phenotype="second whorl sepaloidy,  third whorl missing or carpeloid", @linked_to="pi">, #<Gene:0x000055ee866eea68 @id="AT5G20240", @name="pi", @mutant_phenotype="second whorl sepals,  third whorl carpels", @linked_to="ufo">]